In [164]:
#Import files module
import pandas as pd
import csv
import re                     ##Regular expressions
import glob
import os
import logging
import numpy as np
import datetime
import statsmodels.api as sm
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from math import sqrt
from numpy import concatenate

from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA                     ## ARIMA MODULE
import warnings
from pandas import datetime
from sklearn.metrics import mean_squared_error
import numpy

In [165]:
#logging mechanism for debugging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [166]:
#Mapping months to numeric values
months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'June','07':'July','08':'Aug','09':'Sept','10':'Oct','11':'Nov','12':'Dec'}

In [167]:
import pandas as pd

#Importing files from specific folders and list of holidays
bus_no='6080'                                              #Route number - Change this value accordingly 
path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Up'     #Up Direction files folder Path - Change the path accordingly    
down_path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Down'     #Down Direction files folder path - Change the path accordingly 
down_allfiles=glob.glob(down_path + "/*.csv")
allfiles = glob.glob(path + "/*.csv")
Holiday_colnames = ['Holiday', 'Date', 'Day', 'Month']
Holiday_data = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays.csv', names=Holiday_colnames)    #Holiday list for 2018
Holiday_2017 = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays 2017.csv', names=Holiday_colnames)     #Holiday list for 2017
feature_names =  ['Time Slot','Trips','Passengers','Day','isHoliday']

# Up Direction Processing

In [168]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']
holiday_day_2017 = Holiday_2017['Day']
holiday_month_2017 = Holiday_2017['Month']

In [169]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
import datetime
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,23):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][29:]
    date = os.path.splitext(base)[0][26:28]

    month = os.path.splitext(base)[0][23:25]
    year = os.path.splitext(base)[0][18:22]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [171]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

pass_forecast=[]
for j in xrange(len(timeslot)):
    if(j==19):
        continue
    # load dataset
    series = df[j].copy()
    #print series
    series.drop(['Time Slot','Trips','isHoliday','Day'],axis=1,inplace=True)
    # seasonal difference
    X = series.values
    days_in_year = 7   #Change name
    differenced = difference(X, days_in_year)
    # fit model
    differenced = differenced.astype('float64')
    model = ARIMA(differenced, order=(1,0,0))
    model_fit = model.fit(disp=0)
    # multi-step out-of-sample forecast
    forecast = model_fit.forecast(steps=1)[0]
    #prediction = model_fit.predict(start = 39, end=40)
    # invert the differenced forecast to something usable
    #print prediction
    history = [x for x in X]
    #prediction = inverse_difference(history, prediction, days_in_year)
    #print prediction
    #history = [x for x in X]
    day = 1
    for yhat in forecast:
        inverted = inverse_difference(history, yhat, days_in_year)
        print('Day %d: %f' % (day, inverted))
        pass_forecast.append(round(inverted[0]))
        history.append(inverted)
        day += 1
pass_forecast.append(0.0)


Day 1: -0.084607
Day 1: 0.973699
Day 1: 12.201872
Day 1: 23.951180
Day 1: 14.638834
Day 1: 6.567077
Day 1: 13.561413
Day 1: 4.654739
Day 1: 14.259668
Day 1: 34.286689
Day 1: 30.999855
Day 1: 26.874589
Day 1: 39.747777
Day 1: 61.063561
Day 1: 129.759381
Day 1: 190.155980
Day 1: 147.909105
Day 1: 84.876543
Day 1: 87.908431


In [172]:
#print df[6]
print pass_forecast
print len(pass_forecast)

[-0.0, 1.0, 12.0, 24.0, 15.0, 7.0, 14.0, 5.0, 14.0, 34.0, 31.0, 27.0, 40.0, 61.0, 130.0, 190.0, 148.0, 85.0, 88.0, 0.0]
20


In [173]:
up_trips=[]
for j in range(len(timeslot)):
    if(j==19):
        continue
    target = pd.Series([current_dow,isCurrDay_holiday], index = ['Day','isHoliday'])
    target['Passengers'] = pass_forecast[j]
    df_try=df[j][['Day','isHoliday','Passengers']]
    labels=df[j]['Trips']
    labels=np.asarray(labels,dtype=float)
    df_try=np.asarray(df_try,dtype=float)
    model1 = ARIMA(endog=labels,exog=df_try, order=(1,0,0)).fit()
    trip_forecast =model1.forecast(steps = 1,exog=target)
    if(trip_forecast[0]<0.0001):
        trips=0
    else:
        trips=round(trip_forecast[0])
    up_trips.append(trips)
    print j+1
    print trip_forecast
up_trips.append(0.0)

Day           4.0
isHoliday     0.0
Passengers   -0.0
dtype: float64
     Day  isHoliday  Passengers
0      7          1           0
1      1          1           0
2      2          0          12
3      3          0           0
4      4          0           0
5      5          0           0
6      6          0           0
7      7          0           0
8      1          0           0
9      2          0           0
10     3          0           0
11     4          0           0
12     5          0           0
13     6          0           0
14     7          0           0
15     1          0           0
16     2          0           0
17     3          0           0
18     4          1           0
19     5          1           0
20     6          0           0
21     7          0           0
22     1          0           0
23     2          0           0
24     3          0           0
25     4          0           0
26     5          0           0
27     6          0           0
28 

4
(array([ 5.75004965]), array([ 1.15987305]), array([[ 3.47674024,  8.02335906]]))
Day            4.0
isHoliday      0.0
Passengers    15.0
dtype: float64
     Day  isHoliday  Passengers
0      7          1          16
1      1          1          27
2      2          0          23
3      3          0          27
4      4          0          35
5      5          0          27
6      6          0          37
7      7          0          23
8      1          0          31
9      2          0          48
10     3          0          22
11     4          0          32
12     5          0          38
13     6          0          31
14     7          0          35
15     1          0          23
16     2          0          28
17     3          0          20
18     4          1          27
19     5          1          24
20     6          0          18
21     7          0          26
22     1          0          28
23     2          0          22
24     3          0          22
25     4    

8
(array([ 2.15771549]), array([ 0.8811019]), array([[ 0.4307875 ,  3.88464348]]))
Day            4.0
isHoliday      0.0
Passengers    14.0
dtype: float64
     Day  isHoliday  Passengers
0      7          1          16
1      1          1          32
2      2          0          27
3      3          0          25
4      4          0          36
5      5          0          43
6      6          0          33
7      7          0          55
8      1          0           4
9      2          0          49
10     3          0           0
11     4          0          29
12     5          0          46
13     6          0          35
14     7          0          24
15     1          0          42
16     2          0          38
17     3          0          34
18     4          1          42
19     5          1          51
20     6          0          76
21     7          0          66
22     1          0           7
23     2          0          18
24     3          0          39
25     4     

12
(array([ 3.06709126]), array([ 1.02215204]), array([[ 1.06371008,  5.07047244]]))
Day            4.0
isHoliday      0.0
Passengers    40.0
dtype: float64
     Day  isHoliday  Passengers
0      7          1          41
1      1          1          59
2      2          0          72
3      3          0          40
4      4          0          55
5      5          0          54
6      6          0         155
7      7          0          55
8      1          0         125
9      2          0          91
10     3          0          70
11     4          0          87
12     5          0         101
13     6          0         151
14     7          0          16
15     1          0          27
16     2          0          82
17     3          0          44
18     4          1         113
19     5          1         101
20     6          0         134
21     7          0          49
22     1          0          60
23     2          0          44
24     3          0          60
25     4   

16
(array([ 4.22149832]), array([ 0.91249191]), array([[ 2.43304704,  6.00994961]]))
Day             4.0
isHoliday       0.0
Passengers    148.0
dtype: float64
     Day  isHoliday  Passengers
0      7          1          70
1      1          1          70
2      2          0         150
3      3          0         107
4      4          0         150
5      5          0         205
6      6          0         198
7      7          0         126
8      1          0         188
9      2          0         227
10     3          0         230
11     4          0         217
12     5          0         240
13     6          0          94
14     7          0         125
15     1          0         160
16     2          0         149
17     3          0         119
18     4          1         137
19     5          1         119
20     6          0          76
21     7          0         137
22     1          0         131
23     2          0         134
24     3          0         162
25     4

C:\Users\Lenovo\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


17
(array([ 2.69225512]), array([ 0.95382047]), array([[ 0.82280135,  4.56170889]]))
Day            4.0
isHoliday      0.0
Passengers    85.0
dtype: float64
     Day  isHoliday  Passengers
0      7          1          24
1      1          1         126
2      2          0         149
3      3          0         155
4      4          0         146
5      5          0         138
6      6          0         291
7      7          0          88
8      1          0         233
9      2          0         260
10     3          0         275
11     4          0         233
12     5          0         228
13     6          0         217
14     7          0         111
15     1          0         223
16     2          0         184
17     3          0         207
18     4          1          52
19     5          1         100
20     6          0          28
21     7          0          51
22     1          0         197
23     2          0         210
24     3          0         252
25     4   

# Down Direction Processing

In [175]:
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']

In [176]:
import datetime
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,24):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in down_allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][31:]
    date = os.path.splitext(base)[0][29:31]
    month = os.path.splitext(base)[0][26:28]
    
    #Determining whether holiday or not
    year = os.path.splitext(base)[0][21:25]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [177]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

pass_down=[]
for j in xrange(len(timeslot)):
    # load dataset
    if(j==19):
        continue
    series = df[j].copy()
    #print series
    series.drop(['Time Slot','Trips','isHoliday','Day'],axis=1,inplace=True)
    # seasonal difference
    X = series.values
    days_in_year = 7   #Change name
    differenced = difference(X, days_in_year)
    # fit model
    differenced = differenced.astype('float64')
    model = ARIMA(differenced, order=(1,0,0))
    model_fit = model.fit(disp=0)
    # multi-step out-of-sample forecast
    forecast = model_fit.forecast(steps=1)[0]
    #prediction = model_fit.predict(start = 39, end=40)
    # invert the differenced forecast to something usable
    #print prediction
    history = [x for x in X]
    #prediction = inverse_difference(history, prediction, days_in_year)
    #print prediction
    #history = [x for x in X]
    day = 1
    for yhat in forecast:
        inverted = inverse_difference(history, yhat, days_in_year)
        print('Day %d: %f' % (day, inverted))
        pass_down.append(round(inverted[0]))
        history.append(inverted)
        day += 1
    #pass_down[19] ko 0 kar
pass_down.append(0.0)



Day 1: -0.100617
Day 1: 8.137344
Day 1: 62.732098
Day 1: 78.498818
Day 1: 88.869362
Day 1: 26.658237
Day 1: 45.577433
Day 1: 35.957903
Day 1: 13.009900
Day 1: 31.622706
Day 1: 18.637319
Day 1: 26.197446
Day 1: 32.923462
Day 1: 12.436071
Day 1: 20.725249
Day 1: 20.385559
Day 1: 29.528019
Day 1: 18.919334
Day 1: 23.618070


In [178]:
print pass_down

[-0.0, 8.0, 63.0, 78.0, 89.0, 27.0, 46.0, 36.0, 13.0, 32.0, 19.0, 26.0, 33.0, 12.0, 21.0, 20.0, 30.0, 19.0, 24.0, 0.0]


In [179]:
down_trips=[]
def is_invertible(a):
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

for j in range(len(timeslot)):
    if(j==19):
        continue
    target = pd.Series([current_dow,isCurrDay_holiday], index = ['Day','isHoliday'])
    target['Passengers'] = pass_down[j]
    df_try=df[j][['Day','isHoliday','Passengers']]
    df_try=df_try.as_matrix()
    labels=df[j]['Trips']
    print target
    labels=np.asarray(labels,dtype=float)
    df_try=np.asarray(df_try,dtype=float)
    print df_try
    if(is_invertible(df_try) or df_try.sum()==0 or j==19):
        trips=0
    else:
        model = ARIMA(endog=labels,exog=df_try, order=(1,0,0)).fit(trend='nc')
        trip_forecast =model.forecast(steps = 1,exog=target)
        if(trip_forecast[0]<=0):
            trips=0
        else:
            trips=round(trip_forecast[0])
    down_trips.append(trips)
    print trip_forecast
down_trips.append(0.0)


Day           4.0
isHoliday     0.0
Passengers   -0.0
dtype: float64
[[   7.    0.    0.]
 [   1.    0.    0.]
 [   2.    0.   21.]
 [   3.    0.    0.]
 [   4.    0.    0.]
 [   5.    0.    0.]
 [   6.    0.    0.]
 [   7.    0.    0.]
 [   1.    0.    0.]
 [   2.    0.    0.]
 [   3.    0.    0.]
 [   4.    0.    0.]
 [   5.    0.    0.]
 [   6.    0.    0.]
 [   7.    0.    0.]
 [   1.    0.    0.]
 [   2.    0.    0.]
 [   3.    0.    0.]
 [   4.    0.    3.]
 [   5.    0.    0.]
 [   6.    0.    0.]
 [   7.    0.    0.]
 [   1.    0.    0.]
 [   2.    0.    0.]
 [   3.    0.    0.]
 [   4.    0.    0.]
 [   5.    0.    0.]
 [   6.    0.    0.]
 [   7.    0.    0.]
 [   1.    0.   32.]
 [   2.    0.    0.]
 [   3.    0.    0.]
 [   4.    0.    0.]
 [   5.    0.    0.]
 [   6.    0.   18.]
 [   7.    0.   52.]
 [   1.    0.    0.]
 [   2.    0.    0.]
 [   3.    0.   32.]
 [   4.    0.    0.]
 [   5.    0.    0.]
 [   6.    0.    0.]
 [   7.    0.    0.]
 [   1.    0.    0.]
 [   2.

(array([ 4.78990411]), array([ 1.0106315]), array([[ 2.80910276,  6.77070546]]))
Day            4.0
isHoliday      0.0
Passengers    78.0
dtype: float64
[[   7.    0.   60.]
 [   1.    0.   88.]
 [   2.    0.  204.]
 [   3.    0.  183.]
 [   4.    0.  193.]
 [   5.    0.  149.]
 [   6.    0.  153.]
 [   7.    0.   87.]
 [   1.    0.  223.]
 [   2.    0.  204.]
 [   3.    0.  223.]
 [   4.    0.  212.]
 [   5.    0.  182.]
 [   6.    0.  169.]
 [   7.    0.   73.]
 [   1.    0.  172.]
 [   2.    0.  149.]
 [   3.    0.   83.]
 [   4.    0.   49.]
 [   5.    0.   71.]
 [   6.    0.   79.]
 [   7.    0.   19.]
 [   1.    0.  175.]
 [   2.    0.  152.]
 [   3.    0.  128.]
 [   4.    0.  171.]
 [   5.    0.  153.]
 [   6.    0.  102.]
 [   7.    0.   92.]
 [   1.    0.  165.]
 [   2.    0.  191.]
 [   3.    0.  163.]
 [   4.    0.  155.]
 [   5.    0.  168.]
 [   6.    0.  119.]
 [   7.    0.   33.]
 [   1.    0.  149.]
 [   2.    0.  117.]
 [   3.    0.  125.]
 [   4.    0.  138.]
 [   5.

(array([ 2.27742418]), array([ 2.0483265]), array([[-1.737222  ,  6.29207035]]))
Day            4.0
isHoliday      0.0
Passengers    46.0
dtype: float64
[[   7.    0.   73.]
 [   1.    0.   94.]
 [   2.    0.  111.]
 [   3.    0.   73.]
 [   4.    0.   73.]
 [   5.    0.  112.]
 [   6.    0.  172.]
 [   7.    0.   72.]
 [   1.    0.  127.]
 [   2.    0.   65.]
 [   3.    0.  137.]
 [   4.    0.  194.]
 [   5.    0.   76.]
 [   6.    0.  112.]
 [   7.    0.   69.]
 [   1.    0.   92.]
 [   2.    0.  123.]
 [   3.    0.  115.]
 [   4.    0.   50.]
 [   5.    0.   60.]
 [   6.    0.  102.]
 [   7.    0.   95.]
 [   1.    0.  112.]
 [   2.    0.  157.]
 [   3.    0.   99.]
 [   4.    0.   64.]
 [   5.    0.   97.]
 [   6.    0.   65.]
 [   7.    0.   97.]
 [   1.    0.   93.]
 [   2.    0.   70.]
 [   3.    0.   89.]
 [   4.    0.  120.]
 [   5.    0.   73.]
 [   6.    0.   66.]
 [   7.    0.   82.]
 [   1.    0.   62.]
 [   2.    0.   71.]
 [   3.    0.   68.]
 [   4.    0.   72.]
 [   5.

(array([ 1.31827014]), array([ 1.10172667]), array([[-0.84107445,  3.47761473]]))
Day            4.0
isHoliday      0.0
Passengers    32.0
dtype: float64
[[  7.   0.  34.]
 [  1.   0.  38.]
 [  2.   0.  33.]
 [  3.   0.  43.]
 [  4.   0.  52.]
 [  5.   0.  51.]
 [  6.   0.  25.]
 [  7.   0.  80.]
 [  1.   0.  53.]
 [  2.   0.  93.]
 [  3.   0.   0.]
 [  4.   0.  94.]
 [  5.   0.  43.]
 [  6.   0.  27.]
 [  7.   0.  35.]
 [  1.   0.  38.]
 [  2.   0.  39.]
 [  3.   0.  30.]
 [  4.   0.  41.]
 [  5.   0.  33.]
 [  6.   0.  45.]
 [  7.   0.  26.]
 [  1.   0.  11.]
 [  2.   0.  18.]
 [  3.   0.  29.]
 [  4.   0.  57.]
 [  5.   0.  21.]
 [  6.   0.  17.]
 [  7.   0.  41.]
 [  1.   0.  34.]
 [  2.   0.  39.]
 [  3.   0.  44.]
 [  4.   0.  27.]
 [  5.   0.  45.]
 [  6.   0.  53.]
 [  7.   0.  22.]
 [  1.   0.  49.]
 [  2.   0.  47.]
 [  3.   0.  40.]
 [  4.   0.  43.]
 [  5.   0.  66.]
 [  6.   0.  47.]
 [  7.   0.  36.]
 [  1.   0.  37.]
 [  2.   0.  29.]
 [  3.   0.  24.]
 [  4.   0.  38.]


 [  3.   0.  12.]]
(array([ 3.21119477]), array([ 1.05562125]), array([[ 1.14221514,  5.2801744 ]]))
Day            4.0
isHoliday      0.0
Passengers    12.0
dtype: float64
[[  7.   0.  58.]
 [  1.   0.  40.]
 [  2.   0.  27.]
 [  3.   0.  22.]
 [  4.   0.  25.]
 [  5.   0.  19.]
 [  6.   0.  47.]
 [  7.   0.  33.]
 [  1.   0.  27.]
 [  2.   0.  25.]
 [  3.   0.  21.]
 [  4.   0.  24.]
 [  5.   0.  12.]
 [  6.   0.  20.]
 [  7.   0.  24.]
 [  1.   0.  21.]
 [  2.   0.  17.]
 [  3.   0.  26.]
 [  4.   0.  41.]
 [  5.   0.  24.]
 [  6.   0.  87.]
 [  7.   0.  62.]
 [  1.   0.  31.]
 [  2.   0.  35.]
 [  3.   0.  36.]
 [  4.   0.  40.]
 [  5.   0.  32.]
 [  6.   0.  21.]
 [  7.   0.  45.]
 [  1.   0.  36.]
 [  2.   0.  16.]
 [  3.   0.  31.]
 [  4.   0.  21.]
 [  5.   0.  17.]
 [  6.   0.  12.]
 [  7.   0.  36.]
 [  1.   0.  12.]
 [  2.   0.  27.]
 [  3.   0.  27.]
 [  4.   0.  34.]
 [  5.   0.  17.]
 [  6.   0.  17.]
 [  7.   0.  36.]
 [  1.   0.  27.]
 [  2.   0.  41.]
 [  3.   0.  20.]

(array([ 3.34749983]), array([ 1.29028612]), array([[ 0.8185855 ,  5.87641416]]))
Day            4.0
isHoliday      0.0
Passengers    19.0
dtype: float64
[[  7.   0.   6.]
 [  1.   0.  31.]
 [  2.   0.  29.]
 [  3.   0.  11.]
 [  4.   0.  18.]
 [  5.   0.  37.]
 [  6.   0.  38.]
 [  7.   0.  45.]
 [  1.   0.  25.]
 [  2.   0.  17.]
 [  3.   0.  29.]
 [  4.   0.  27.]
 [  5.   0.  28.]
 [  6.   0.  30.]
 [  7.   0.  28.]
 [  1.   0.  39.]
 [  2.   0.  54.]
 [  3.   0.  27.]
 [  4.   0.  35.]
 [  5.   0.  25.]
 [  6.   0.  30.]
 [  7.   0.  40.]
 [  1.   0.  31.]
 [  2.   0.  13.]
 [  3.   0.  32.]
 [  4.   0.  26.]
 [  5.   0.  32.]
 [  6.   0.  25.]
 [  7.   0.  33.]
 [  1.   0.  25.]
 [  2.   0.  17.]
 [  3.   0.  24.]
 [  4.   0.  14.]
 [  5.   0.  26.]
 [  6.   0.  15.]
 [  7.   0.  11.]
 [  1.   0.  23.]
 [  2.   0.  13.]
 [  3.   0.  19.]
 [  4.   0.  47.]
 [  5.   0.  24.]
 [  6.   0.  28.]
 [  7.   0.  37.]
 [  1.   0.  18.]
 [  2.   0.  48.]
 [  3.   0.  12.]
 [  4.   0.  25.]


In [180]:
print len(down_trips)
print len(pass_down)
print len(pass_forecast)
print len(up_trips)

20
20
20
20


In [181]:
#Saving Results into CSV files 
forecasted_trips = pd.DataFrame(columns=['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass'])
save_path=r'D:/Dharmit/Final Year Project/Forecasted Trips/'
for j in xrange(len(timeslot)):
    forecasted_trips= forecasted_trips.append(pd.Series([timeslot[j],up_trips[j],pass_forecast[j],down_trips[j],pass_down[j]], index = ['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass']), ignore_index=True)
filename ='Forecasted_trips_'+bus_no+'_'+str(now.month).zfill(2) +'_'+str(now.day)+'_'+str(day)+'_ARIMA'
forecasted_trips.to_csv(save_path + filename +'.csv')